***RAN ON P100-PCIE-16GB AND T4 GPUs***

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-29ef0ce8-8d8a-0853-e483-09946b7ea9bc)


# Install Dependencies, Here I have used a wonderful library *SIMPLE TRANSFORMERS* by ThilinaRajapakse.I have used this as a baseline and then changed some of its Internal code.The new code will be provided.

In [ ]:
#Install Dependencies:(Run as it is)
!pip install tensorboardX --quiet
!git clone 'https://github.com/ThilinaRajapakse/simpletransformers.git'

!pip uninstall apex 
!git clone https://github.com/NVIDIA/apex

! cd apex
!pip install -v --no-cache-dir ./apex --quiet

     |████████████████████████████████| 317kB 8.1MB/s 
Cloning into 'simpletransformers'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 6333 (delta 1), reused 2 (delta 0), pack-reused 6326
Receiving objects: 100% (6333/6333), 18.71 MiB | 14.68 MiB/s, done.
Resolving deltas: 100% (4007/4007), done.
Cloning into 'apex'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 7424 (delta 6), reused 4 (delta 1), pack-reused 7400
Receiving objects: 100% (7424/7424), 13.92 MiB | 2.01 MiB/s, done.
Resolving deltas: 100% (5005/5005), done.
Processing ./apex
  Created wheel for apex: filename=apex-0.1-cp36-none-any.whl size=192840 sha256=59a8bf9f1ecf7258f60442860d526647eb0933241cee38c330f339273264355e
  Stored in directory: /tmp/pip-ephem-wheel-cache-jpar4fod/wheels/b1/3a/aa/d84906eaab780ae580c7a5686a33bf

# **Change the file in simpletransformers>simpletransformers>classification>classification_model.py with what we provided so that it may work in exact manner we want.**

In [ ]:
#After Above changes is done Nows lets Install this awesome library
!cd simpletransformers
!pip install -v --no-cache-dir ./simpletransformers --quiet


#We also need to import some libraries:
import torch
import gc
import pandas as pd
from simpletransformers.classification import ClassificationModel
import logging
import sklearn

import warnings
warnings.simplefilter('ignore')

Processing ./simpletransformers
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 778kB 16.3MB/s 
     |████████████████████████████████| 3.0MB 31.5MB/s 
     |████████████████████████████████| 1.4MB 53.8MB/s 
     |████████████████████████████████| 7.1MB 57.0MB/s 
     |████████████████████████████████| 1.1MB 60.2MB/s 
     |████████████████████████████████| 890kB 45.6MB/s 
     |████████████████████████████████| 112kB 57.0MB/s 
     |████████████████████████████████| 102kB 50.4MB/s 
     |████████████████████████████████| 102kB 62.3MB/s 
     |████████████████████████████████| 163kB 64.3MB/s 
     |████████████████████████████████| 4.4MB 50.4MB/s 
     |████████████████████████████████| 112kB 60.8MB/s 
     |████████████████████████████████| 71kB 54.7MB/s 
     |████████████████████████████████| 71kB 58.4MB/s 
     |████████████████████████████████| 122kB 62.9MB/s 
  Created wheel for simpletransformers: filename=simpletransformers-0.47.0-c

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


# **Text Preprocessing has played a lot more role in this dataset.The one which we haven't cleaned outperformed in this competition.The Gap was very clear..... **
# Cleaned dataset scored around 2-3% less than uncleaned dataset.So we used the same dataset as provided and gave it directly to model

In [ ]:
#Download the datasets from these urls..

train_url = 'https://raw.githubusercontent.com/MiHarsh/MiHarsh/master/train_folds_final.csv'
test_url = 'https://raw.githubusercontent.com/dranzerblaze/kaggle_fakenewskdd20/master/Data/test.csv'


df = pd.read_csv(train_url ,sep = ',')
df_test = pd.read_csv(test_url ,sep = '\t')


In [ ]:
ppp=pd.read_csv('/content/pseudo_pakka_uncleaned.csv')
ppp.head(5)

,text,label
0,The 2017 Teen Choice Awards ceremony was held ...,0.0
1,This is worse than a lump of coal in your stoc...,1.0
2,Luann De Lesseps is going to rehab after her a...,0.0
3,Diamond's Weinstein meets a grisly end Saved ...,0.0
4,"After 13 years, Ashlee Simpson-Ross is once ag...",0.0


This dataset has very long long texts.In train the max_len was around 13000 and in test it was around 10000.Generally advanced models such as Bert/Roberta has limitation for max_len.

To overcome this problem we used a truncation techniques in which we had to simply truncated long texts into a bunch of short texts (max_len and num_new_words was a hyperparameter) with labels same as long_texts.

When we did this We had a problem that it can be the case that the previous short text was responsible for the fakeness of the Whole long text.

We then introduced a method in which the long texts will get truncated in such a manner that it contains some context from the previous short text.This method gave us significant boost(scored 81.4% from 0.80% with same hyperparameters.

In [ ]:
#This function will consider 'new_words' new words and  ('max_len'-'new_words')words from the previous context...In this way I will
#keep a check on dependency...

def split_text(new_words,max_len,text,label=None):
    bag=[]
    chunks=[]
    if len(text.split())//new_words >0:
        n=len(text.split())//new_words
    else:
        n=1
    for w in range(n):
        if w==0:
            chunks=text.split()[:new_words]
            if label is None:
                bag.append(" ".join(chunks))
            else:
                bag.append([" ".join(chunks),label])
        else:
            chunks=text.split()[w*new_words:w*new_words+max_len]
            if label is None:
                bag.append(" ".join(chunks))
            else:
                bag.append([" ".join(chunks),label])
    return bag

**Now comes the trainer Part.Advanced models have a problem of overfitting.To overcome and check for those what I did is implemented a logger and also get predictions from each epoch.This also helped us in ensembling.**

In [ ]:
# This TrainKDD function requires--
# 1)Fold number
# 2)df-the train_csv file
# 3)test_df-perform testing on every epoch
# 4)new_words-number of new texts
# 5)max_len-max_length
# 6)learning rate-lr (We experimented with a lot lrs and found 2e-05 to be the best for this.)
# 7)epochs-number of epochs to train
# 8)pseudo-this is pseudo labelling,It has been done based on the top validation accuracies and the label to be given was found by
#  hard voting method.If all the labels of a particular example are 0 or 1 then only they are considered here in pseudo label.

In [ ]:
def TrainKDD(fold,df,test_df,pseudo,new_words,max_len,learning_rate,epochs):
    #Split in folds:
    # train_df=df[df.kfold!=fold].drop(['kfold'],axis=1).reset_index(drop=True)
    # val_df=df[df.kfold==fold].drop(['kfold'],axis=1).reset_index(drop=True)
    train_df=df.drop(['kfold'],axis=1)
    concat=pd.concat([train_df,pseudo],axis=0,join='inner')
    train_df=concat
    val_df=df[df.kfold==fold].drop(['kfold'],axis=1).reset_index(drop=True)
    # concat=pd.concat([train_df,pseudo],axis=0,join='inner')
    # concat.sample(frac=1)

    #Split the train on the hypothesis:
    print('Splitting Data Based on new words and max_len')
    dats=[]
    for i in range(len(train_df)):
        text=train_df.iloc[i].text
        label=train_df.iloc[i].label
        dats.extend(split_text(new_words,max_len,text,label))
    train=pd.DataFrame(dats)
    train=train.rename({0:'text',1:'label'},axis=1)
    print('Done....')
    
    #Model Args:
    model_args = {
    'evaluate_during_training' : False , 
    'learning_rate' : learning_rate ,
    'num_train_epochs': epochs , 
    'reprocess_input_data': True , 
    'use_early_stopping': False ,
    'evaluate_during_training_steps' : 300 , 
    'max_seq_length' : max_len,
    'overwrite_output_dir':True
    # 'sliding_window':True
    }

    #Lets Define Model:
    model = ClassificationModel('roberta', 'roberta-base',args  = model_args)

    #Lets Train our model for given strategies:
    model.train_model(train_df,eval_df=val_df,test_df=test_df,acc = sklearn.metrics.accuracy_score)

    #After training is done...
    # torch.cuda.empty_cache()
    # del dats
    # del model
    # gc.collect()




In [ ]:
TrainKDD(0,df,df_test,ppp,128,150,2e-5,7)

Splitting Data Based on new words and max_len
Done....


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out


Train_loss0.5266453514397746,Val_loss=0.3821867856681347,Accuracy=0.8326653306613226,lr=1.8242523555919707e-05
Testing it too.......




Train_loss0.4547169230297218,Val_loss=0.24789070284366607,Accuracy=0.8997995991983968,lr=1.5202785743547727e-05
Testing it too.......




Train_loss0.3909811634250924,Val_loss=0.12363967907056213,Accuracy=0.9579158316633266,lr=1.2163047931175749e-05
Testing it too.......




Train_loss0.32817758626061067,Val_loss=0.07320866433810443,Accuracy=0.9779559118236473,lr=9.123310118803769e-06
Testing it too.......




Train_loss0.27543521145170174,Val_loss=0.03542737458017655,Accuracy=0.9879759519038076,lr=6.083572306431791e-06
Testing it too.......




Train_loss0.23534799348123686,Val_loss=0.02083297783782473,Accuracy=0.9919839679358717,lr=3.043834494059812e-06
Testing it too.......




Train_loss0.20467582391583244,Val_loss=0.02134372361021815,Accuracy=0.9909819639278558,lr=4.096681687832856e-09
Testing it too.......

